In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
import re


In [6]:
from extract_dataframe import read_json         # a function to load json_data 
from extract_dataframe import TweetDfExtractor  # and a class to extract relevant variables.    
from clean_tweets_dataframe import Clean_Tweets        # collection of functions to for cleaning the dataframe.
import dashboard_manager as ds_m   # imports a list of functions to create a dashboard
import database_manager as db_m  # a list of functions to create a database, create tables, and manage data input-output.

ImportError: cannot import name 'read_json' from 'extract_dataframe' (C:\Users\User\anaconda3\envs\10_10\extract_dataframe.py)

In [ ]:
_, tweet_list = read_json("data/.json")
tweet = TweetDfExtractor(tweet_list)
tweet.get_tweet_df(True)  # this will also generate a CSV file.
print("Please, load the CSV file")